In [1]:
import os
import cv2
import json
import torch
import pyclipper
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from hydra import initialize, compose
from albumentations.pytorch import ToTensorV2

from runners import predict
from ocr.datasets.base import OCRDataset

In [2]:
device = torch.device("cuda")

In [3]:
with initialize(config_path="configs"):
    cfg = compose("predict.yaml")

model_module, data_module, temp = predict.predict(cfg)

/tmp/ipykernel_304914/2759098359.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="configs"):
Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at ./outputs/ocr_training_2024-04-25_11-00-48/checkpoints/epoch=7-step=18552.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./outputs/ocr_training_2024

Predicting: |          | 0/? [00:00<?, ?it/s]

In [32]:
data_module.dataset['predict'][0]['image']

tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -1

In [ ]:
data_module.dataset['predict'][0]['image']

tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -1

In [34]:
data_module.dataset['predict'][0]['image'].unsqueeze(0).shape

torch.Size([1, 3, 640, 640])

In [ ]:
data_module.dataset['predict'][0]['image'].shape

torch.Size([3, 640, 640])

In [25]:
#@title 추론 결과 후처리를 위한 Class

class DBPostProcessor:
    def __init__(self, thresh=0.3, box_thresh=0.4, max_candidates=300, use_polygon=False):
        self.min_size = 3                       # 텍스트 검출 최소 사이즈
        self.thresh = thresh                    # 모델의 출력 결과 Segmentation map을 Binarization 하기 위한 Threshold
        self.box_thresh = box_thresh            # 텍스트 영역 판정을 위한 Threshold
        self.max_candidates = max_candidates    # 최대 텍스트 검출 후보 영역의 갯수
        self.use_polygon = use_polygon          # Polygon 검출을 할지 Quad 검출을 할지에 대한 Type flag

    # Segmentation map 결과로부터 텍스트 영역의 좌표를 반환하는 메소드
    def represent(self, batch, _pred):
        """
        batch: a dict produced by dataloaders.
            images: tensor of shape (N, C, H, W).
            polygons: tensor of shape (N, K, 4, 2), the polygons of objective regions.
            ignore_tags: tensor of shape (N, K), indicates whether a region is ignorable or not.
            shape: the original shape of images.
            inverse_matrix: Warp Perspective Matrix, with shape (3, 3) as NDArray[float32]
            filename: the original filenames of images.
        pred:
            prob_maps: text region segmentation map, with shape (N, 1, H, W)
        """
        # assert 'images' in batch is not None, "images is required in batch"
        images = batch['image']

        # Use prob_maps if pred is a dict
        if isinstance(_pred, dict):
            assert 'prob_maps' in _pred is not None, "prob_maps is required in _pred"
            pred = _pred['prob_maps']
        else:
            pred = _pred

        assert 'inverse_matrix' in batch is not None, "inverse_matrix is required in batch"
        inverse_matrix = batch['inverse_matrix']

        # Probability map을 이진화
        segmentation = self.binarize(pred)

        boxes_batch = []
        scores_batch = []
        for batch_index in range(images.size(0)):
            if self.use_polygon:
                # Get polygons from segmentation
                boxes, scores = self.polygons_from_bitmap(
                                        pred[batch_index],
                                        segmentation[batch_index],
                                        inverse_matrix=inverse_matrix[batch_index])
            else:
                # Get boxes from segmentation
                boxes, scores = self.boxes_from_bitmap(
                                        pred[batch_index],
                                        segmentation[batch_index],
                                        inverse_matrix=inverse_matrix[batch_index])
            # Append to batch
            boxes_batch.append(boxes)
            scores_batch.append(scores)

        return boxes_batch, scores_batch

    @staticmethod
    # Inverse matrix를 이용하여 원본 이미지에 대한 좌표로 변환 하는 메소드
    def __transform_coordinates(coords, matrix):
        """
        Transform coordinates according to the warp matrix

        coords: (N, 2) as NDArray[float32]
        matrix: (3, 3) as NDArray[float32]
        return: (N, 2) as NDArray[float32]
        """
        coords = np.array(coords)
        coords = np.dot(matrix, np.vstack([coords.T, np.ones(coords.shape[0])]))
        coords /= coords[2, :]
        return coords.T[:, :2]

    # Binarize the prediction
    def binarize(self, pred):
        # 후처리 과정에서 단일 Threshold를 사용하여 이진화
        return pred > torch.Tensor([self.thresh]).to(device=pred.device)

    # 추론 결과 영역에서 텍스트 영역을 검출하여 반환하는 메소드 (Polygon으로 추출할때)
    def polygons_from_bitmap(self, pred, _bitmap,
                             inverse_matrix=None):
        """
        Extracts polygons and their scores from a bitmap image.

        _bitmap: single map with shape (1, H, W),
            whose values are binarized as {0, 1}
        """

        assert _bitmap.size(0) == 1
        bitmap = _bitmap.cpu().numpy()[0]  # The first channel
        pred = pred.cpu().detach().numpy()[0]

        boxes = []
        scores = []

        # 컨투어 영역 찾기
        # Find contours from the binarized map
        # contours: a list of contours
        # https://docs.opencv.org/4.9.0/d4/d73/tutorial_py_contours_begin.html
        contours, _ = cv2.findContours(
            (bitmap * 255).astype(np.uint8),
            cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # 최대 텍스트 검출 후보 영역의 갯수만큼 컨투어 영역이 텍스트 영역인지 판단
        # Get the top N contours
        for contour in contours[:self.max_candidates]:
            # 복잡한 컨투어 픽셀을 근사화하여 영역 다듬기
            # Approximate the contour with Douglas-Peucker algorithm
            # https://docs.opencv.org/4.9.0/dc/dcf/tutorial_js_contour_features.html
            epsilon = 0.002 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            points = approx.reshape((-1, 2))
            # 최소 포인트가 4개 미만인 경우 무시
            if points.shape[0] < 4:
                continue

            # 컨투어 영역에서 텍스트가 차지하는 영역에 대한 점수 계산
            # Get the score of the box
            score = self.box_score_fast(pred, points.reshape(-1, 2))
            if self.box_thresh > score:
                continue

            # Shrink되어있는 추론 결과를 다시 Dilate하는 과정
            # Unclip the box
            if points.shape[0] > 2:
                box = self.unclip(points, unclip_ratio=2.0)
                if box is None:
                    continue
            else:
                continue

            # 주어진 영역의 크기를 구하고 최소 사이즈 미만의 경우 무시
            # Get the mini box
            box = box.reshape(-1, 2)
            _, sside = self.get_mini_boxes(box.reshape((-1, 1, 2)))
            if sside < self.min_size + 2:
                continue

            # Inverse Matrix를 이용하여 원본 이미지의 위치로 좌표 변환
            # Transform the coordinates
            box = self.__transform_coordinates(box, inverse_matrix)

            # Append to the list
            boxes.append(np.round(box).astype(np.int16).tolist())
            scores.append(score)

        return boxes, scores

    # 추론 결과 영역에서 텍스트 영역을 검출하여 반환하는 메소드 (QuadBox로 추출할때)
    # 텍스트 영역을 찾는 과정을 제외하고 Polygon과 동일
    def boxes_from_bitmap(self, pred, _bitmap,
                          inverse_matrix=None):
        """
        Extracts bounding boxes and their scores from a bitmap image.

        _bitmap: single map with shape (1, H, W),
            whose values are binarized as {0, 1}
        """

        assert _bitmap.size(0) == 1
        bitmap = _bitmap.cpu().numpy()[0]  # The first channel
        pred = pred.cpu().detach().numpy()[0]

        boxes = []
        scores = []

        # 컨투어 영역 찾기
        # Find contours from the binarized map
        # contours: a list of contours
        # https://docs.opencv.org/4.9.0/d4/d73/tutorial_py_contours_begin.html
        contours, _ = cv2.findContours(
            (bitmap * 255).astype(np.uint8),
            cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        num_contours = min(len(contours), self.max_candidates)

        # 최대 텍스트 검출 후보 영역의 갯수만큼 컨투어 영역이 텍스트 영역인지 판단
        # Get the top N contours
        for index in range(num_contours):
            # 컨투어 영역의 좌표와 크기를 구하고 최소 사이즈 이하의 경우 무시
            # Get the mini box
            contour = contours[index]
            points, sside = self.get_mini_boxes(contour)
            if sside < self.min_size:
                continue

            # 컨투어 영역에서 텍스트가 차지하는 영역에 대한 점수 계산
            # Get the score of the box
            points = np.array(points)
            score = self.box_score_fast(pred, points.reshape(-1, 2))
            if self.box_thresh > score:
                continue

            # Shrink되어있는 추론 결과를 다시 Dilate하는 과정
            # Unclip the box
            box = self.unclip(points).reshape(-1, 1, 2)
            box, sside = self.get_mini_boxes(box)
            if sside < self.min_size + 2:
                continue
            box = np.array(box)

            # Inverse Matrix를 이용하여 원본 이미지의 위치로 좌표 변환
            # Transform the coordinates
            box = self.__transform_coordinates(box, inverse_matrix)

            # Append to the list
            boxes.append(np.round(box).astype(np.int16).tolist())
            scores.append(score)

        return boxes, scores

    def unclip(self, box, unclip_ratio=1.5):
        """
        Expands the given box by a specified ratio.

        box: a list of points of shape (N, 2)
        unclip_ratio: the ratio of unclipping the box
        return: a list of points of shape (N, 2)
        """

        # transform the box to polygon
        poly = Polygon(box)
        if poly.area == 0 or poly.length == 0:
            return None

        # get the expanded polygon
        # unclip_ratio의 비율로 확대된 Distance를 구하고 영역을 확장함
        distance = poly.area * unclip_ratio / poly.length
        offset = pyclipper.PyclipperOffset()
        offset.AddPath(box, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
        expanded = np.array(offset.Execute(distance)[0])

        return expanded

    def get_mini_boxes(self, contour):
        """
        Converts a contour into its minimum area bounding box.

        contour: a list of points of shape (N, 1, 2)
        return: a list of points of shape (N, 2)
        """

        # Get the bounding box
        # https://docs.opencv.org/4.9.0/de/d62/tutorial_bounding_rotated_ellipses.html
        bounding_box = cv2.minAreaRect(contour)
        points = sorted(list(cv2.boxPoints(bounding_box)), key=lambda x: x[0])

        index_1, index_2, index_3, index_4 = 0, 1, 2, 3
        if points[1][1] > points[0][1]:
            index_1 = 0
            index_4 = 1
        else:
            index_1 = 1
            index_4 = 0
        if points[3][1] > points[2][1]:
            index_2 = 2
            index_3 = 3
        else:
            index_2 = 3
            index_3 = 2

        box = [points[index_1], points[index_2],
               points[index_3], points[index_4]]
        return box, min(bounding_box[1])

    def box_score_fast(self, bitmap, _box):
        """
        Calculates a score for a box in a bitmap.
        The score is the percentage of the box area that overlaps with
        the highlighted areas (marked as 1) in the bitmap.

        bitmap: a single map with shape (H, W), whose values are binarized as {0, 1}
        _box: a list of points of shape (N, 2)
        return: a score of the box as float32
        """

        h, w = bitmap.shape[:2]
        box = _box.copy()
        xmin = np.clip(np.floor(box[:, 0].min()).astype(np.int32), 0, w - 1)
        xmax = np.clip(np.ceil(box[:, 0].max()).astype(np.int32), 0, w - 1)
        ymin = np.clip(np.floor(box[:, 1].min()).astype(np.int32), 0, h - 1)
        ymax = np.clip(np.ceil(box[:, 1].max()).astype(np.int32), 0, h - 1)

        mask = np.zeros((ymax - ymin + 1, xmax - xmin + 1), dtype=np.uint8)
        box[:, 0] = box[:, 0] - xmin
        box[:, 1] = box[:, 1] - ymin

        cv2.fillPoly(mask, box.reshape(1, -1, 2).astype(np.int32), 1)

        return cv2.mean(bitmap[ymin:ymax + 1, xmin:xmax + 1], mask)[0]

In [26]:
post_processor = DBPostProcessor()

In [43]:
temp[0]['prob_maps'].shape

torch.Size([1, 640, 640])

In [40]:
temp[0]['prob_maps'].squeeze(0).shape

torch.Size([1, 640, 640])

In [41]:
for i in range(len(temp)):
    temp[i]['prob_maps'] = temp[i]['prob_maps'].squeeze(0)

In [42]:
boxes_batch, scores_batch = post_processor.represent(data_module.dataset['predict'][0], temp[0])

AssertionError: 